<a href="https://colab.research.google.com/github/JasonArmitage-res/Lab_Multimodal_ML/blob/master/TextUnimodal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
import pandas as pd
!pip install --upgrade tables

dataset_embeddings = pd.read_hdf('/content/drive/My Drive/dataset/mm_imdb_embeddings.h5', 'embeddings')

In [0]:
import torch
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split


def Train_Test_Split(data , test_data_fraction = 0.2) :
    
    mlb = MultiLabelBinarizer()
    data_genres_one_hot_encoding = mlb.fit_transform(data['genres'])
    Label_names = mlb.classes_
    data_genres_one_hot_encoding = pd.DataFrame(data_genres_one_hot_encoding, columns = mlb.classes_)
    Data_train, Data_test, Labels_train, Labels_test = train_test_split(data, data_genres_one_hot_encoding, test_size = test_data_fraction)
    Labels_train = torch.tensor(Labels_train.values)
    Labels_test = torch.tensor(Labels_test.values)
    
    Data_train = Data_train.reset_index(drop=True)
    Data_test = Data_test.reset_index(drop=True)

    return (Data_train, Data_test, Labels_train, Labels_test, Label_names)
    
Data_train, Data_test, Labels_train_tensor, Labels_test_tensor, Label_names = Train_Test_Split(dataset_embeddings)

In [0]:
#For text unimodal classifier
import numpy as np

Data_train_tensor = torch.tensor(Data_train['bert_embeddings'])
Data_test_tensor = torch.tensor(Data_test['bert_embeddings'])

In [0]:
!pip install transformers
from transformers import BertModel
from torch import nn


class BertMultiLabelClassifier(nn.Module):

    def __init__(self, hidden_layer_size = 512, hidden_activation = "tanh", input_size = 768, num_labels = 23, dropout = 0.1, gen_embeddings = False):

        super(BertMultiLabelClassifier, self).__init__()
        self.num_labels = num_labels
        self.input_size = input_size
        self.gen_embeddings = gen_embeddings
        self.hidden_layer_size = hidden_layer_size

        self.base_model = BertModel.from_pretrained('bert-base-uncased')
        
        self.hidden_layer = torch.nn.Linear(self.input_size, self.hidden_layer_size)
        
        if(hidden_activation == "tanh") :
          self.hidden_activation = torch.nn.Tanh()
        elif(hidden_activation == "relu") :
          self.hidden_activation = torch.nn.ReLU()
        elif(hidden_activation == "sigmoid") :
          self.hidden_activation = torch.nn.Sigmoid()
        elif(hidden_activation == "leaky_relu") :
          self.hidden_activation = torch.nn.LeakyReLU()
        else :
          return("Invalid hidden_activation parameter value.")

        self.output_layer = torch.nn.Linear(self.hidden_layer_size, self.num_labels)
        self.output_activation = torch.nn.Sigmoid()

        self.dropout = nn.Dropout(p=dropout)

    def forward(self, indexed_tokens, segment_ids=None, masked_ids=None):
        
        if(self.gen_embeddings):
            pooled_output = self.base_model(indexed_tokens, segment_ids, masked_ids)
            embeddings = pooled_output[1]
            return embeddings
        else :
            embeddings = indexed_tokens
        
        logits1 = self.hidden_layer(embeddings)
        activation1 = self.hidden_activation(logits1)

        dropped = self.dropout(activation1)

        logits2 = self.output_layer(dropped)
        if(self.training) :
            return logits2
        else :
            output = self.output_activation(logits2)
            return output


    def freeze_base_model(self):
        for param in self.base_model.parameters():
            param.requires_grad = False


    def unfreeze_base_model(self):
        for param in self.base_model.named_parameters():
            param.requires_grad = True

In [0]:
import torch
from torch.utils.data import DataLoader, SequentialSampler
from transformers import BertConfig

def GetTextEmbeddings_Bert(dataset, batch_size = 32):
  
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  input_ids = TextTransformation_Bert(dataset['plot'])
  sampler = SequentialSampler(input_ids)
  dataloader = DataLoader(input_ids, sampler=sampler, batch_size=batch_size)
  
  results = torch.Tensor().to(device)

  #config = BertConfig.from_pretrained('bert-base-uncased')
  model = BertMultiLabelClassifier(gen_embeddings = True)
  model.to(device)
  model.eval()

  for num, batch_data in enumerate(dataloader):
      print(num) 
      indexed_tokens, segment_ids , masked_ids = tuple(t for t in batch_data)   
      data = indexed_tokens.to(device)
      with torch.no_grad():
        emdeddings = model(data)
      results = torch.cat((results, emdeddings))

  return results

In [0]:
import nltk
from transformers import BertTokenizer
from nltk import tokenize
#nltk.download('punkt')
from torch.utils.data import Dataset

class TextTransformation_Bert(Dataset) :

    def __init__(self, text_list, max_input_length = 512):

        self.text_list = text_list
        self.indexed_tokens = []
        self.segment_ids = []
        self.masked_ids = []
        self.max_input_length = max_input_length

    def GetIndexedTokens(self, text):
        
        tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        tokenized_text = tokenizer.tokenize(text)
        tokenized_text.append("[SEP]")
        tokenized_text.insert(0,"[CLS]")
        self.indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
        

    def GetSegmentIds(self) :
        
        self.segment_ids = [1] * len(self.indexed_tokens)

 
    def GetMaskedIds(self) :
        
        self.masked_ids = [1] * len(self.indexed_tokens)


    def Padding(self) :

        if(len(self.indexed_tokens) < self.max_input_length) :
           padding = [0]*(self.max_input_length - len(self.indexed_tokens))
           self.indexed_tokens += padding
           self.segment_ids += padding
           self.masked_ids += padding
        else :
           del self.indexed_tokens[self.max_input_length:]
           del self.segment_ids[self.max_input_length:]
           del self.masked_ids[self.max_input_length:]

    def __len__(self):
        return len(self.text_list)


    def __getitem__(self, idx):

        text = self.text_list[idx]

        self.GetIndexedTokens(text)
        self.GetSegmentIds()
        self.GetMaskedIds()
        self.Padding()
        
        self.indexed_tokens = torch.tensor(self.indexed_tokens)
        self.segment_ids = torch.tensor(self.segment_ids)
        self.masked_ids = torch.tensor(self.masked_ids)
        
        return self.indexed_tokens, self.segment_ids , self.masked_ids

In [0]:
import torch
from torch.nn import BCEWithLogitsLoss
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from tqdm import tqdm, trange
!pip install transformers
from transformers import BertConfig, AdamW, get_linear_schedule_with_warmup
from sklearn import metrics
import matplotlib.pyplot as plt
% matplotlib inline
import seaborn as sns
import pandas as pd
import time
import datetime

'''
USAGE :

1. If text_unimodal = True
   Data_train_tensor , Data_test_tensor : Image embeddings as tensors of shape [#train_samples, 4096], [#test_samples, 4096]

2. If image_unimodal = True
   Data_train_tensor , Data_test_tensor : Text embeddings as tensors of shape [#train_samples, 768], [#test_samples, 768]


3. Labels_train_tensor, Labels_test_tensor : one-hot encoded labels as tensors of shape [#train_samples, 23], [#test_samples, 23]

'''

class Training_Testing():

    def __init__(self, Data_train_tensor, Labels_train_tensor, Data_test_tensor, Labels_test_tensor, hidden_layer_size = 512,
                 Label_names = None, freeze_base_model = True, image_unimodal = False, text_unimodal = False,
                 hidden_activation = "tanh", batch_size = 32, epochs = 10, sigmoid_thresh = 0.2, learning_rate = 2e-5, num_labels = 23, dropout = 0.1):

      
      #self.dropout = dropout
      #self.hidden_layer_size = hidden_layer_size
      #self.hidden_activation = hidden_activation

      if(text_unimodal) :
        #self.config = BertConfig.from_pretrained('bert-base-uncased')
        self.model = BertMultiLabelClassifier(hidden_layer_size = hidden_layer_size, hidden_activation = hidden_activation, dropout = dropout).cuda()

      
      elif(image_unimodal) :
        self.model = VGG16MultiLabelClassifier(hidden_layer_size = hidden_layer_size, hidden_activation = hidden_activation, dropout = dropout).cuda()
 

      if(freeze_base_model) :
        self.model.freeze_base_model()
      self.label_names = Label_names
      self.num_labels = num_labels
      self.batch_size = batch_size
      self.learning_rate = learning_rate
      self.epochs = epochs
      self.sigmoid_thresh = sigmoid_thresh
      self.optimizer = self.SetOptimizer()
      self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
      self.results = pd.DataFrame(0, index=['Recall','Precision','F_Score'], columns=['micro', 'macro', 'weighted', 'samples']).astype(float)
      self.epoch_loss_set = []
      self.train_dataloader = self.SetTrainDataloader(Data_train_tensor, Labels_train_tensor)
      self.test_dataloader = self.SetTestDataloader(Data_test_tensor, Labels_test_tensor)
      self.scheduler = self.SetScheduler()


    def SetOptimizer(self) :

      optimizer = AdamW(self.model.parameters(), self.learning_rate, eps = 1e-6)
      return(optimizer)

    

    def SetScheduler(self) :

      scheduler = get_linear_schedule_with_warmup(self.optimizer, num_warmup_steps = 0, 
                                                  num_training_steps = self.epochs*len(self.train_dataloader))
      return(scheduler) 



    def Get_Metrics(self, actual, predicted) :

      #acc = metrics.accuracy_score(actual, predicted)
      #hamming = metrics.hamming_loss(actual, predicted)
      #(metrics.roc_auc_score(actual, predicted, average=average)
      averages = ('micro', 'macro', 'weighted', 'samples')
      for average in averages:
          precision, recall, fscore, _ = metrics.precision_recall_fscore_support(actual, predicted, average=average)
          self.results[average]['Recall'] += recall
          self.results[average]['Precision'] += precision
          self.results[average]['F_Score'] += fscore


    #source: https://mccormickml.com/2019/07/22/BERT-fine-tuning/
    def Plot_Training_Epoch_Loss(self) :

      sns.set(style='darkgrid')
      sns.set(font_scale=1.5)
      plt.rcParams["figure.figsize"] = (12,6)
      plt.plot(self.epoch_loss_set, 'b-o')
      plt.title("Training loss")
      plt.xlabel("Epoch")
      plt.ylabel("Loss")
      plt.savefig('Training_Epoch_Loss.png',bbox_inches='tight')
      plt.show()



    #source: https://mccormickml.com/2019/07/22/BERT-fine-tuning/
    def format_time(self, elapsed):
      '''
      Takes a time in seconds and returns a string hh:mm:ss
      '''
      # Round to the nearest second.
      elapsed_rounded = int(round((elapsed)))
      return str(datetime.timedelta(seconds=elapsed_rounded))


    def SetTrainDataloader(self, Data_train_tensor, Labels_train_tensor) :

      train_dataset = TensorDataset(Data_train_tensor, Labels_train_tensor)
      train_sampler = RandomSampler(train_dataset)
      train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size = self.batch_size)
      return(train_dataloader)


    def SetTestDataloader(self, Data_test_tensor, Labels_test_tensor) :
      
      test_dataset = TensorDataset(Data_test_tensor, Labels_test_tensor)
      test_sampler = SequentialSampler(test_dataset)
      test_dataloader = DataLoader(test_dataset, sampler=test_sampler, batch_size = Data_test_tensor.shape[0])
      return(test_dataloader)


    def Train(self) :

      for _ in trange(self.epochs, desc="Epoch"):
        
        self.model.train()
        epoch_loss = 0

        # Measure how long the training epoch takes.
        t0 = time.time()
    
        for step_num, batch_data in enumerate(self.train_dataloader):

          # Progress update every 30 batches.
          if step_num % 30 == 0 and not step_num == 0:
            elapsed = self.format_time(time.time() - t0)
            print('  Batch : ',step_num, ' , Time elapsed : ',elapsed)

          samples, labels = tuple(t.to(self.device) for t in batch_data)
          self.optimizer.zero_grad()
          logits = self.model(samples.float())
          loss_fct = BCEWithLogitsLoss()
          batch_loss = loss_fct(logits.view(-1, self.num_labels).float(), labels.view(-1, self.num_labels).float())
          batch_loss.backward()
          self.optimizer.step()
          self.scheduler.step()
          epoch_loss += batch_loss.item()

        avg_epoch_loss = epoch_loss/len(self.train_dataloader)
        print("\nTrain loss for epoch: ",avg_epoch_loss)
        print("\nTraining epoch took: {:}".format(self.format_time(time.time() - t0)))
        self.epoch_loss_set.append(avg_epoch_loss)

      torch.save(self.model.state_dict(), "/content/drive/My Drive/dataset/model.pt")
      self.Plot_Training_Epoch_Loss()
    

    def Test(self) :

      # Put model in evaluation mode to evaluate loss on the test set
      self.model.eval()

      for batch_data in self.test_dataloader:
  
        samples, labels = tuple(t.to(self.device) for t in batch_data)
      
        # Telling the model not to compute or store gradients, saving memory and speeding up validation
        # Forward pass, calculate logit predictions
        with torch.no_grad():
          output = self.model(samples)

        threshold = torch.Tensor([self.sigmoid_thresh]).to(self.device)
        predictions = (output > threshold).int()

        # Move preds and labels to CPU
        predictions = predictions.detach().cpu().numpy()
        labels = labels.to('cpu').numpy()
      
        self.Get_Metrics(labels, predictions)
    
      self.results = self.results/len(self.test_dataloader)
      #print("Test data metrics : \n")
      
      return(self.results)

    

In [0]:
train_test = Training_Testing(Data_train_tensor, Labels_train_tensor, Data_test_tensor, Labels_test_tensor, Label_names=Label_names
                              hidden_layer_size=768, hidden_activation = "relu", text_unimodal = True, epochs = 200, batch_size= 256, learning_rate=0.001, dropout=0.1, sigmoid_thresh = 0.2)
train_test.Train()
train_test.Test()

In [0]:
'''
import os
import shutil

results_dir = "/content/drive/My Drive/results"
model_dir = results_dir + "/TextUnimodal"
current_dir = os.getcwd()





#For testing varying epochs :
def Test_epochs(Data_train_tensor, Labels_train_tensor, Data_test_tensor, Labels_test_tensor, 
                model_dir, current_dir, 
                hidden_layer_size = 512, hidden_activation = "tanh", batch_size= 512, learning_rate=0.01, dropout=0.1, sigmoid_thresh = 0.2,
                epochs = [100, 150, 200]) :

  
  epochs_dir = model_dir + "/Epochs"
  if(os.path.exists(epochs_dir)) :
     shutil.rmtree(epochs_dir)
  
  os.mkdir(epochs_dir)


  params_dict = {'hidden_layer_size' : hidden_layer_size, 'hidden_activation' : hidden_activation, 'batch_size' : batch_size, 'learning_rate' : learning_rate, 'dropout' : dropout, 'sigmoid_thresh' : sigmoid_thresh}
  with open(epochs_dir + '/other_params.txt', 'w') as f:
    print(params_dict, file=f)

  results = pd.DataFrame(index=['Recall','Precision','F_Score']).astype(float)

  for i in epochs :

      sub_dir = epochs_dir + '/Epochs_' + str(i)
      os.mkdir(sub_dir)
      os.chdir(sub_dir)

      train_test = Training_Testing(Data_train_tensor, Labels_train_tensor, Data_test_tensor, Labels_test_tensor, text_unimodal = True, 
                                    hidden_layer_size = hidden_layer_size, hidden_activation = hidden_activation, epochs = i, 
                                    batch_size = batch_size, learning_rate = learning_rate, dropout = dropout, sigmoid_thresh = sigmoid_thresh)
      train_test.Train()
      result = train_test.Test()
      print('\n\n',result,'\n\n')
      results['Epochs = ' + str(i)] = result['micro']

      os.chdir(current_dir)
  
  results.to_excel(epochs_dir + "/Micro_metrics.xlsx", header = True, index = True)







#For testing varying batch sizes :
def Test_batch_size(Data_train_tensor, Labels_train_tensor, Data_test_tensor, Labels_test_tensor, 
                    model_dir, current_dir, 
                    hidden_layer_size = 512, hidden_activation = "tanh", epochs = 100, learning_rate=0.01, dropout=0.1, sigmoid_thresh = 0.2,
                    batch_size = [64, 128, 256, 512]) :

  
  batch_dir = model_dir + "/BatchSizes"
  if(os.path.exists(batch_dir)) :
     shutil.rmtree(batch_dir)
  
  os.mkdir(batch_dir)

  params_dict = {'hidden_layer_size' : hidden_layer_size, 'hidden_activation' : hidden_activation, 'epochs' : epochs, 'learning_rate' : learning_rate, 'dropout' : dropout, 'sigmoid_thresh' : sigmoid_thresh}
  with open(batch_dir + '/other_params.txt', 'w') as f:
    print(params_dict, file=f)

  results = pd.DataFrame(index=['Recall','Precision','F_Score']).astype(float)

  for i in batch_size :

      sub_dir = batch_dir + '/BatchSize_' + str(i)
      os.mkdir(sub_dir)
      os.chdir(sub_dir)

      train_test = Training_Testing(Data_train_tensor, Labels_train_tensor, Data_test_tensor, Labels_test_tensor, text_unimodal = True, 
                                    hidden_layer_size = hidden_layer_size, hidden_activation = hidden_activation, epochs = epochs, 
                                    batch_size = i, learning_rate = learning_rate, dropout = dropout, sigmoid_thresh = sigmoid_thresh)
      train_test.Train()
      result = train_test.Test()
      print('\n\n',result,'\n\n')
      results['Batch Size = ' + str(i)] = result['micro']

      os.chdir(current_dir)
  
  results.to_excel(batch_dir + "/Micro_metrics.xlsx", header = True, index = True)






#For testing varying dropout rates :
def Test_dropout_rate(Data_train_tensor, Labels_train_tensor, Data_test_tensor, Labels_test_tensor, 
                       model_dir, current_dir, 
                       hidden_layer_size = 512, hidden_activation = "tanh", epochs = 100, batch_size = 512, learning_rate = 0.01, sigmoid_thresh = 0.2,
                       dropout = [0.1 , 0.2, 0.5]) :

  
  dropout_rate_dir = model_dir + "/DropoutRate"
  if(os.path.exists(dropout_rate_dir)) :
     shutil.rmtree(dropout_rate_dir)
  
  os.mkdir(dropout_rate_dir)

  params_dict = {'hidden_layer_size' : hidden_layer_size, 'hidden_activation' : hidden_activation, 'epochs' : epochs, 'batch_size' : batch_size, 'learning_rate' : learning_rate, 'sigmoid_thresh' : sigmoid_thresh}
  with open(dropout_rate_dir + '/other_params.txt', 'w') as f:
    print(params_dict, file=f)

  results = pd.DataFrame(index=['Recall','Precision','F_Score']).astype(float)

  for i in dropout :

      sub_dir = dropout_rate_dir + '/DropoutRate_' + str(i)
      os.mkdir(sub_dir)
      os.chdir(sub_dir)

      train_test = Training_Testing(Data_train_tensor, Labels_train_tensor, Data_test_tensor, Labels_test_tensor, text_unimodal = True, 
                                    hidden_layer_size = hidden_layer_size, hidden_activation = hidden_activation, epochs = epochs, 
                                    batch_size = batch_size, learning_rate = learning_rate, dropout = i, sigmoid_thresh = sigmoid_thresh)
      train_test.Train()
      result = train_test.Test()
      print('\n\n',result,'\n\n')
      results['Dropout Rate = ' + str(i)] = result['micro']

      os.chdir(current_dir)
  
  results.to_excel(dropout_rate_dir + "/Micro_metrics.xlsx", header = True, index = True)





#For testing varying learning rates :
def Test_learning_rate(Data_train_tensor, Labels_train_tensor, Data_test_tensor, Labels_test_tensor, 
                       model_dir, current_dir, 
                       hidden_layer_size = 512, hidden_activation = "tanh", epochs = 100, batch_size = 512, dropout=0.1, sigmoid_thresh = 0.2,
                       learning_rate = [0.1 , 0.05, 0.01, 0.001, 0.0001, 0.00001]) :

  
  learning_rate_dir = model_dir + "/LearningRate"
  if(os.path.exists(learning_rate_dir)) :
     shutil.rmtree(learning_rate_dir)
  
  os.mkdir(learning_rate_dir)

  params_dict = {'hidden_layer_size' : hidden_layer_size, 'hidden_activation' : hidden_activation, 'epochs' : epochs, 'batch_size' : batch_size, 'dropout' : dropout, 'sigmoid_thresh' : sigmoid_thresh}
  with open(learning_rate_dir + '/other_params.txt', 'w') as f:
    print(params_dict, file=f)

  results = pd.DataFrame(index=['Recall','Precision','F_Score']).astype(float)

  for i in learning_rate :

      sub_dir = learning_rate_dir + '/LearningRate_' + str(i)
      os.mkdir(sub_dir)
      os.chdir(sub_dir)

      train_test = Training_Testing(Data_train_tensor, Labels_train_tensor, Data_test_tensor, Labels_test_tensor, text_unimodal = True, 
                                    hidden_layer_size = hidden_layer_size, hidden_activation = hidden_activation, epochs = epochs, 
                                    batch_size = batch_size, learning_rate = i, dropout = dropout, sigmoid_thresh = sigmoid_thresh)
      train_test.Train()
      result = train_test.Test()
      print('\n\n',result,'\n\n')
      results['Learning Rate = ' + str(i)] = result['micro']

      os.chdir(current_dir)
  
  results.to_excel(learning_rate_dir + "/Micro_metrics.xlsx", header = True, index = True)






#For testing varying activation functions :
def Test_activation_fn(Data_train_tensor, Labels_train_tensor, Data_test_tensor, Labels_test_tensor, 
                       model_dir, current_dir, 
                       hidden_layer_size = 512, learning_rate = 0.01, epochs = 100, batch_size = 512, dropout=0.1, sigmoid_thresh = 0.2,
                       hidden_activation = ["tanh", "relu", "sigmoid", "leaky_relu"]) :

  
  activation_fn_dir = model_dir + "/ActivationFunc"
  if(os.path.exists(activation_fn_dir)) :
     shutil.rmtree(activation_fn_dir)
  
  os.mkdir(activation_fn_dir)

  params_dict = {'hidden_layer_size' : hidden_layer_size, 'learning_rate' : learning_rate, 'epochs' : epochs, 'batch_size' : batch_size, 'dropout' : dropout, 'sigmoid_thresh' : sigmoid_thresh}
  with open(activation_fn_dir + '/other_params.txt', 'w') as f:
    print(params_dict, file=f)

  results = pd.DataFrame(index=['Recall','Precision','F_Score']).astype(float)

  for i in hidden_activation :

      sub_dir = activation_fn_dir + '/ActivationFunc_' + i
      os.mkdir(sub_dir)
      os.chdir(sub_dir)

      train_test = Training_Testing(Data_train_tensor, Labels_train_tensor, Data_test_tensor, Labels_test_tensor, text_unimodal = True, 
                                    hidden_layer_size = hidden_layer_size, hidden_activation = i, epochs = epochs, 
                                    batch_size = batch_size, learning_rate = learning_rate, dropout = dropout, sigmoid_thresh = sigmoid_thresh)
      train_test.Train()
      result = train_test.Test()
      print('\n\n',result,'\n\n')
      results['Activation Function = ' + i] = result['micro']

      os.chdir(current_dir)
  
  results.to_excel(activation_fn_dir + "/Micro_metrics.xlsx", header = True, index = True)






#For testing varying #hidden neurons :
def Test_hidden_neurons(Data_train_tensor, Labels_train_tensor, Data_test_tensor, Labels_test_tensor, 
                       model_dir, current_dir, 
                       hidden_activation = "tanh" , learning_rate = 0.01, epochs = 100, batch_size = 512, dropout = 0.1, sigmoid_thresh = 0.2,
                       hidden_layer_size = [512, 256]) :

  
  hidden_neurons_dir = model_dir + "/HiddenNeurons"
  if(os.path.exists(hidden_neurons_dir)) :
     shutil.rmtree(hidden_neurons_dir)
  
  os.mkdir(hidden_neurons_dir)

  params_dict = {'hidden_activation' : hidden_activation, 'learning_rate' : learning_rate, 'epochs' : epochs, 'batch_size' : batch_size, 'dropout' : dropout, 'sigmoid_thresh' : sigmoid_thresh}
  with open(hidden_neurons_dir + '/other_params.txt', 'w') as f:
    print(params_dict, file=f)

  results = pd.DataFrame(index=['Recall','Precision','F_Score']).astype(float)

  for i in hidden_layer_size :

      sub_dir = hidden_neurons_dir + '/HiddenNeurons_' + str(i)
      os.mkdir(sub_dir)
      os.chdir(sub_dir)

      train_test = Training_Testing(Data_train_tensor, Labels_train_tensor, Data_test_tensor, Labels_test_tensor, text_unimodal = True, 
                                    hidden_layer_size = i, hidden_activation = hidden_activation, epochs = epochs, 
                                    batch_size = batch_size, learning_rate = learning_rate, dropout = dropout, sigmoid_thresh = sigmoid_thresh)
      train_test.Train()
      result = train_test.Test()
      print('\n\n',result,'\n\n')
      results['#Hidden Neurons = ' + str(i)] = result['micro']

      os.chdir(current_dir)
  
  results.to_excel(hidden_neurons_dir + "/Micro_metrics.xlsx", header = True, index = True)







#For testing varying sigmoid threshold :
def Test_sigmoid_threshold(Data_train_tensor, Labels_train_tensor, Data_test_tensor, Labels_test_tensor, 
                       model_dir, current_dir, 
                       hidden_activation = "tanh" , learning_rate = 0.01, epochs = 100, batch_size = 512, dropout = 0.1, hidden_layer_size = 512,
                       sigmoid_thresh = [0.15, 0.18, 0.2, 0.23, 0.25]) :

  
  sigmoid_threshold_dir = model_dir + "/SigmoidThreshold"
  if(os.path.exists(sigmoid_threshold_dir)) :
     shutil.rmtree(sigmoid_threshold_dir)
  
  os.mkdir(sigmoid_threshold_dir)

  params_dict = {'hidden_activation' : hidden_activation, 'learning_rate' : learning_rate, 'epochs' : epochs, 'batch_size' : batch_size, 'dropout' : dropout, 'sigmoid_thresh' : sigmoid_thresh}
  with open(sigmoid_threshold_dir + '/other_params.txt', 'w') as f:
    print(params_dict, file=f)

  results = pd.DataFrame(index=['Recall','Precision','F_Score']).astype(float)

  for i in sigmoid_thresh :

      sub_dir = sigmoid_threshold_dir + '/SigmoidThreshold_' + str(i)
      os.mkdir(sub_dir)
      os.chdir(sub_dir)

      train_test = Training_Testing(Data_train_tensor, Labels_train_tensor, Data_test_tensor, Labels_test_tensor, text_unimodal = True, 
                                    hidden_layer_size = hidden_layer_size, hidden_activation = hidden_activation, epochs = epochs, 
                                    batch_size = batch_size, learning_rate = learning_rate, dropout = dropout, sigmoid_thresh = i)
      train_test.Train()
      result = train_test.Test()
      print('\n\n',result,'\n\n')
      results['Sigmoid Threshold = ' + str(i)] = result['micro']

      os.chdir(current_dir)
  
  results.to_excel(sigmoid_threshold_dir + "/Micro_metrics.xlsx", header = True, index = True)
  '''

In [0]:
'''
Test_epochs(Data_train_tensor, Labels_train_tensor, Data_test_tensor, Labels_test_tensor, model_dir, current_dir, 
            hidden_layer_size = 768, hidden_activation = "relu", batch_size= 256, learning_rate=0.001, dropout=0.1, sigmoid_thresh = 0.2, epochs = [100, 150, 200])

Test_batch_size(Data_train_tensor, Labels_train_tensor, Data_test_tensor, Labels_test_tensor, model_dir, current_dir, 
                hidden_layer_size = 768, hidden_activation = "relu", learning_rate=0.001, dropout=0.1, epochs = 100, sigmoid_thresh = 0.2, batch_size = [256, 512, 1024])

Test_learning_rate(Data_train_tensor, Labels_train_tensor, Data_test_tensor, Labels_test_tensor, model_dir, current_dir, 
                   hidden_layer_size = 768, hidden_activation = "relu", batch_size= 256, dropout=0.1, epochs = 100, sigmoid_thresh = 0.2, learning_rate = [0.1 , 0.05, 0.01, 0.001, 0.0001, 0.00001])

Test_dropout_rate(Data_train_tensor, Labels_train_tensor, Data_test_tensor, Labels_test_tensor, model_dir, current_dir, 
                  hidden_layer_size = 768, hidden_activation = "relu", batch_size= 256, learning_rate=0.001, epochs = 100, sigmoid_thresh = 0.2, dropout = [0.1, 0.5])

Test_activation_fn(Data_train_tensor, Labels_train_tensor, Data_test_tensor, Labels_test_tensor, model_dir, current_dir, 
                   hidden_layer_size = 768, batch_size= 256, learning_rate=0.001, epochs = 100, dropout = 0.1, sigmoid_thresh = 0.2, hidden_activation = ["tanh", "relu", "sigmoid", "leaky_relu"])

Test_hidden_neurons(Data_train_tensor, Labels_train_tensor, Data_test_tensor, Labels_test_tensor, model_dir, current_dir, 
                    hidden_activation = "relu", batch_size= 256, learning_rate=0.001, epochs = 100, dropout = 0.1, sigmoid_thresh = 0.2, hidden_layer_size = [256 , 535, 768, 1000])

Test_sigmoid_threshold(Data_train_tensor, Labels_train_tensor, Data_test_tensor, Labels_test_tensor, model_dir, current_dir, 
                    hidden_activation = "relu", batch_size= 256, learning_rate=0.001, epochs = 100, dropout = 0.1, hidden_layer_size = 768, sigmoid_thresh = [0.15, 0.18, 0.2, 0.23, 0.25])
'''